In [ ]:
import ModelingToolkit as Model
import SymPy as sp
import Symbolics as Symb
using DomainSets
import ApproxFun as AF
using NonlinearSolve
import DifferentialEquations as DE
include("multiharmonic_balance.jl");

Equation parameters

In [ ]:
g = 9.81;
height = 3.0;
omega = 10.0;

Discretization parameters

In [ ]:
harmonics = 1; # number of harmonics
xleft::Float64 = 0.0;
xright::Float64 = 1.0;
Nx = 40;
stepx = (xright-xleft)/Nx;
u0 = zeros((Nx+1) * harmonics * 2);

**Declare the problem**

In [ ]:
# Define symbolics
Model.@parameters x, t;

const Dx = Model.Differential(x);
const Dt = Model.Differential(t);

In [ ]:
vars, var_exprs, fields = create_ansatz((x,), t, omega, harmonics, 2);
u = fields[1];
eta = fields[2];
pdes = [Dt(eta) + height * Dx(u),
       Dt(u) + g * Dx(eta)];

In [ ]:
u

In [ ]:
eta

In [ ]:
vars

In [ ]:
function simplify_problem(pdes, t, omega, harmonics, Nx, vars)
    all_sym_eqs = Expr[]
    
    for pde in pdes
        expanded = expand_trig_jl(pde, t, omega)
        eqns = make_residual(expanded, harmonics, omega, t)
        sym_eqs = map(transform_sym(Nx) ∘ Meta.parse ∘ string, eqns)
        append!(all_sym_eqs, sym_eqs)
    end
    resid = create_residual_function(all_sym_eqs, vars, Nx,
    bc_left = Dict(
        :A => :(A[i]),                    # u = 0
        :B => :(B[i]),                    # u = 0
        :C => :(C[i] - exp(-x^2)),        # η = exp(-x²)
        :D => :(D[i] - exp(-x^2))         # η = exp(-x²)
    )
)
    return resid
end

In [ ]:
@time begin
simplify_problem(pdes, t, omega, harmonics, Nx, vars);
end